# Passo 1: Gerando o Dataset

Vamos começar criando um dataset sintético para ilustrar o problema. Usaremos a função `make_blobs` para criar grupos compactos, que é o cenário ideal para o algoritmo K-Means.

Imagine que estes pontos representam clientes e as coordenadas são variáveis como "Idade" e "Gasto".

### Parâmetros Importantes:
* **`n_samples`**: O número total de observações (ex: 500 clientes).
* **`centers`**: O número real de grupos que vamos gerar. Como estamos simulando um problema não supervisionado, o algoritmo não saberá esse número; nós usaremos isso apenas para verificar se ele acertou no final.
* **`cluster_std`**: O desvio padrão dos clusters. Define o quão "espalhados" ou compactos eles são.
* **`random_state`**: A semente para garantir que seus dados sejam iguais aos meus.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

# Gerar dados simulados (Clusters compactos)
X, y_true = make_blobs(n_samples=500, centers=4, cluster_std=0.60, random_state=0)

# Visualizar os dados "crus" (como o algoritmo vê)
plt.scatter(X[:, 0], X[:, 1], s=20)
plt.title("Dados Originais (Sem Rótulos)")
plt.xlabel("Feature 1")
plt.ylabel("Feature 2")
plt.show()

# Passo 2: Padronização dos Dados

Para algoritmos baseados em distância (como K-Means), a escala das variáveis é crítica. Se uma variável varia entre 0-1000 e outra entre 0-1, a primeira dominará o cálculo da distância Euclidiana.

Para resolver isso, usamos o `StandardScaler` (Z-score), transformando os dados para que tenham média 0 e desvio padrão 1.

### Parâmetros Importantes:
* **`fit_transform(X)`**: Calcula a média e o desvio padrão de cada coluna em `X` e aplica a transformação $z = \frac{x - \mu}{\sigma}$.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Média antes escala:", np.mean(X))
print("Desvio padrão antes escala:", np.std(X))
print("Média após escala:", np.mean(X_scaled))
print("Desvio padrão após escala:", np.std(X_scaled))

# Passo 3: Escolhendo k (Método do Cotovelo)

O K-Means exige que definamos o número de clusters ($k$) antes de rodar. Para descobrir o melhor $k$, usamos o Método do Cotovelo (Elbow Method).

Rodamos o algoritmo para vários valores de $k$ e observamos a **Inércia (SSE)**. O objetivo é encontrar o ponto onde o ganho marginal diminui drasticamente (o "cotovelo").

### Parâmetros Importantes do `KMeans`:
* **`n_clusters`**: O número de grupos ($k$) que estamos testando naquela iteração.
* **`n_init`**: Quantas vezes o algoritmo vai rodar com sementes diferentes. A regra prática sugere executar várias inicializações e ficar com a menor SSE. O padrão do scikit-learn é 10.
* **`random_state`**: Para reprodutibilidade.
* **Atributo `inertia_`**: Retorna a soma dos erros ao quadrado (SSE) do modelo treinado.

In [ ]:
from sklearn.cluster import KMeans

sse = []
k_range = range(1, 11)

for k in k_range:
    kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
    kmeans.fit(X_scaled)
    sse.append(kmeans.inertia_)

# Plotar o gráfico do Cotovelo
plt.figure(figsize=(8, 5))
plt.plot(k_range, sse, marker='o')
plt.title('Método do Cotovelo (Elbow Method)')
plt.xlabel('Número de clusters (k)')
plt.ylabel('Inércia (SSE)')
plt.grid(True)
plt.show()

# Passo 4: Aplicando o K-Means

Analisando o gráfico acima, devemos ver um "cotovelo" claro em $k=4$. A curva achata depois desse ponto. Vamos treinar o modelo final com esse valor.

### Parâmetros Importantes:
* **`n_clusters=4`**: O valor escolhido via método do cotovelo.
* **`fit_predict(X)`**: Executa dois passos:
    1.  Calcula os centróides (treina).
    2.  Atribui cada ponto ao cluster mais próximo (prediz).

In [ ]:
# Treinando com o k ideal
kmeans_final = KMeans(n_clusters=4, n_init=10, random_state=42)
y_kmeans = kmeans_final.fit_predict(X_scaled)

# Visualizando o resultado
plt.scatter(X_scaled[:, 0], X_scaled[:, 1], c=y_kmeans, s=20, cmap='viridis')

# Plotar os centróides
centers = kmeans_final.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=200, alpha=0.7, label='Centróides')
plt.title("Clusterização Final (k=4)")
plt.legend()
plt.show()

# Passo 5: Avaliação com Silhouette Score

Para confirmar se a partição é boa sem usar rótulos externos, usamos o **Silhouette Score**. Ele mede o quão similar um ponto é ao seu próprio cluster em comparação aos outros clusters.

* Valor próximo de +1: Clusters bem definidos e separados.
* Valor próximo de 0: Sobreposição de clusters.
* Valor negativo: Pontos possivelmente no cluster errado.

### Parâmetros Importantes:
* **`silhouette_score(X, labels)`**: Recebe os dados e os rótulos preditos pelo modelo. Retorna a média de todos os pontos.

In [ ]:
from sklearn.metrics import silhouette_score

score = silhouette_score(X_scaled, y_kmeans)
print(f"Silhouette Score para k=4: {score:.3f}")

# Interpretação rápida
if score > 0.5:
    print("Resultado: Clusters bem separados e definidos.")
elif score > 0.2:
    print("Resultado: Separação razoável.")
else:
    print("Resultado: Clusters sobrepostos ou mal definidos.")